# TD 3 : Entrainer un réseau de neurones basique

In [ ]:
# @title Installation
!pip3 uninstall --yes torch torchaudio torchvision torchtext torchdata
!pip3 install torch torchaudio torchvision torchtext torchdata

Dans ce TD, vous allez définir puis entrainer un réseau de neurones très basique capable de reconnaitre les nombres de 0 à 9.

Pour cela, nous allons utiliser le dataset MNIST. MNIST est un petit dataset contenant des chiffres manuscrits.

In [ ]:
import torch
import torchvision
from torch.utils.data import Dataset
from PIL import Image
from IPython.display import display
import numpy as np

Tout d'abord, nous avons besoin de définir quelques outils utiles. Une classe `MNIST_dataset` pour interpréter les données de MNIST comme des tensors pytorch et une fonction `show_elem` pour afficher à l'écran un tensor donné.

In [ ]:
class MNIST_dataset(Dataset):

    def __init__(self, base_data):
        self.base_data = base_data

    def __getitem__(self, idx):
        img, label = self.base_data[idx]
        img = torch.FloatTensor(np.array(img)) / 255
        return img.view(1, 28, 28), label

    def __len__(self):
        return len(self.base_data)

def show_elem(in_item):
    N, H, L = in_item.size()
    s = 112
    in_item*=255
    out = np.zeros((s, s*N), dtype='uint8')
    for n in range(N):
        x = torch.clamp(in_item[n], 0, 255)
        x = x.detach().numpy().astype(np.uint8)
        im = Image.fromarray(x)
        im = im.resize((s, s), resample=0)
        out[:, s*n:(n+1)*s] = np.asarray(im)
    display(Image.fromarray(out))

Nous pouvons à présent charger les datasets de train et de test.

In [ ]:
# Nous avons besoin de deux datasets: un dataset d'entrainement et un de validation
test_dataset = MNIST_dataset(torchvision.datasets.MNIST(root = '/content/MNIST', download= True, train=False))
train_dataset = MNIST_dataset(torchvision.datasets.MNIST(root = '/content/MNIST', download= True, train=True))

Montrons ci-dessous quelques exemples d'images que l'on trouve dans MNIST. Chaque image est associé avec un label qui indique son contenu.

In [ ]:
item_0, label_0 = test_dataset[0]
item_52, label_52 = test_dataset[52]
print(f"LABEL : {label_0}")
show_elem(item_0)

print(f"LABEL : {label_52}")
show_elem(item_52)

## Exercice 1:

Chercher la taille d'une image typique de MNIST. Par exemple la taille de l'image 48 du test_dataset

In [ ]:
# Votre code ici

## Exercice 2:

Vous allez maintenant définir votre réseau.

La classe de base pour tout réseau de neurones dans pytorch est [torch.nn.module](https://pytorch.org/docs/stable/nn.html). Quand vous définissez un réseau de neurones, ce dernier doit hériter de la classe torch.nn.module. Notez qu'un module peut contenir d'autres modules.

Quand vous créer un nouveau modèle, vous devez lui définir une fonction forward qui déterminera les opeérations effectives faites par le modèle.

Construisez un réseau convolutionnel effectuant les opérations suivantes:
- Une convolution de taille 4 prenant une image contenant un canal en entrée et sortant une image avec 4 canaux
- Une activation ReLU
- Une convolution de taille 4 prenant une image contenant 4 canaux en entrée et sortant une image avec 4 canaux
- Une autre activation ReLU
- Un max pooling de taille 2
- Une convertion du batch d'image obtenu, alors de taille BATCH_SIZE x 4 x 4 x 4 en un vecteur de taille BATCH_SIZE x 64
- Une couche Linéaire pleinement connectée sortant un vecteur de dimension 10

N'hésitez pas à utiliser les documentations de [torch.nn.Conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html), [torch.nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html#relu), [torch.nn.MaxPool2d](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html#maxpool2d) et [torch.nn.Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear) pour vous aider.

In [ ]:
# Nous définissons une classe MonReseau qui hérite de torch.nn.module
class MonReseau(torch.nn.Module):

    # En python, __init__ est le constructeur de la classe
    def __init__(self):

        # Lorsque l'on crée un module, il faut toujours commencer par initialiser la classe torch.nn.module
        super(MonReseau, self).__init__()

        # Notre modéla possédera 5 modules
        # VOTRE CODE ICI

    # Un module doit toujours définir une fonction forward
    def forward(self, x):
        N, C, H, L = x.size()

        # VOTRE CODE ici


        return self.classifier(x)         # Renvoie le resultat

## Exercice 3: entrainons un réseau

Vous allez à présent entrainer votre réseau de neurones. Commençons tout d'abord par créer une instance du réseau

In [ ]:
# ATTENTION: NE PAS RELANCER CETTE CELLULE APRES AVOIR EFFECTUE L'ENTRAINEMENT. Vous perdriez tout !
mon_reseau = MonReseau()

mon_reseau est un réseau de neurones qui prend en entrée une image noir et blanc de taille 28x28 et renvoie un vecteur de score de taille 10. Voyez plutôt:

In [ ]:
# Fabriquons une image aléatoire et passons là dans mon_reseau
x = torch.randn(1, 1, 28, 28)
score_x = mon_reseau(x)
print(score_x.size())

Ensuite nous avons besoin de mettre les datasets dans des DataLoader. En pytorch le DataLoader a deux fonctions:

1) Regrouper les images du dataset en batch de plusieurs images

2) Mélanger ces batchs et les donner dans un ordre aléatoire

Fabriquons le DataLoader du dataset d'entrainement:

In [ ]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

Les arguments pour construire un data-loader sont les suivants:

1) dataset que l'on souhaite charger

2) batch_size : taille de chaque batch. Dans l'exemple chaque batch contidendra 4 images

3) shuffle: True si les batchs doivent être mélangés

De la même manière, créez test_loader, le DataLoader de la base de validation. Mais cette fois-ci, donnez lui un batch_size de 5

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=5, shuffle=False)

À présent il nous faut une loss. On utilisera la cross entropy. Dans pytorch, la classe torch.nn.CrossEntropyLoss() combine en un seul module à la fois le softmax et la cross-entropy.

In [ ]:
loss_mode = torch.nn.CrossEntropyLoss()

Pour finir, nous avons besoin d'un optimizer. En pytorch l'optimizer est la classe qui utilise les gradients déjà calculés après la rétro-propagation pour mettre à jour les poids du réseau de neurones.

Nous prenons un SGD (Stochastic Gradient Descent) qui correspond à une descente de gradient classique.

In [ ]:
optimizer = torch.optim.SGD(mon_reseau.parameters(), lr=2e-3)

Le paramètre lr que vous pouvez voir s'appelle le **learning rate**: il correspond à la taille du pas fait lors de la descente de gradient. Plus le **learning rate** est élevé, plus le réseau apprend vite, mais plus ses performances à la convergence seront approximatives.

Nous pouvons à présent entrainer le réseau ! Ça va prendre un peu de temps (environ 5min selon les machines), c'est normal.

In [ ]:
# Nous parcourir 3 fois le dataset d'entrainement, on dit que l'on fait 3 epochs
for epoch in range(3):
    avg_loss_train = 0
    n_steps_train = 0

    # Phase d'entrainement
    # Nous mettons le réseau en mode train()
    mon_reseau.train()

    # On parcourt le dataset d'entrainement
    for data, label in train_loader:

        # On commence par remettre tous les gradients à zéro
        optimizer.zero_grad()

        # Puis on calcule le vecteur de score du réseau
        score = mon_reseau(data)

        # On compare ce vecteur score avec les labels des images pour obetnir une loss
        loss = loss_mode(score, label)

        # À partir de la loss on fait la rétropropagation du gradient.
        # Pytorch devine tout seul les gradients à calculer ! Pas besoin de les lui dire.
        loss.backward()

        # Maintenant que les gradients sont calculés on peut mettre à jour les paramètres du model
        # Cette opération se fait sur l'optimizer avec .step()
        optimizer.step()

        # On met à jour la loss moyenne calculée lors de l'entrainment pour avoir une idée de ce qu'il se
        # passe
        avg_loss_train = avg_loss_train + loss
        n_steps_train = n_steps_train + 1

    print(f"Epoch {epoch} loss train = {avg_loss_train / n_steps_train}")

    # Phase de test
    # Nous mettons le réseau en mode eval()
    mon_reseau.eval()

    avg_loss_test = 0
    n_steps_test = 0

    for data, label in test_loader:

        # Ici pas de gradient: nous voulons juste calculer la loss et la précision du modèle
        score = mon_reseau(data)
        loss = loss_mode(score, label)
        avg_loss_test= avg_loss_test + loss
        n_steps_test =n_steps_test +1

    print(f"Epoch {epoch} loss test = {avg_loss_test / n_steps_test}")

Maintenant nous aimerions savoir si le réseau a appris quelque chose. Faisons un test:

In [ ]:
image_test_3, label_test_3 = test_dataset[3]
image_test_4, label_test_4 = test_dataset[4]
image_test_25, label_test_25 = test_dataset[25]
show_elem(image_test_3)
show_elem(image_test_4)
show_elem(image_test_25)

In [ ]:
# On met les image_test dans un format lisible pour le réseau
image_test_3 = image_test_3.view(1, 1, 28, 28)
image_test_4 = image_test_4.view(1, 1, 28, 28)
image_test_25 = image_test_25.view(1, 1, 28, 28)

# On les concatene en batch
batch_test = torch.cat([image_test_3, image_test_4, image_test_25], dim=0)

1) Calculez score_test le vecteur de score de batch_test donné par mon_reseau()

2) Calculez index_test, certitude_test l'index et la certitudes des predictions (pensez à [torch.nn.functional.softmax](https://pytorch.org/docs/stable/generated/torch.nn.functional.softmax.html))

In [ ]:
# Votre Code ici

## Exercice 3 (BONUS):

Trois exemples ce n'est **pas suffisant** pour savoir si un réseau marche ou non. Pour le savoir, il faut calculer le taux d'erreur sur l'ensemble du dataset de test.

La fonction ci-dessous calcule le taux d'erreur sur un batch:

In [ ]:
def get_error_rate(model, input_batch, input_label):
    r"""
    Calcule le taux d'erreur du modèle donné sur le batch donné.
    Arguments:
    model : le model à appliquer
    input_batch : un batch d'images au format N x C x H x L
    input_label : un tensor de labels au format N
    """
    # On calcule le vecteur de score
    # Score est de taille N x10
    scores = model(input_batch)

    # On cherche l'element du réseau
    # max_index est de taille N
    max_score, max_index = scores.max(dim=1)

    N = max_index.size(0)

    # On regarde toutes les fois ou le score maximal ne correspond pas au label
    #print(max_index, input_label)
    is_false =  max_index != input_label

    # On retourne le taux d'erreur:
    return is_false.sum() / float(N)

Utilisez la fonction get_error_rate pour calculer le taux d'erreur moyen sur l'ensemble du dataset de test

In [ ]:
# Votre code ici